# Imports
We import some neccesary libraries and download some global scope data

In [226]:
import re
import os
import json
import pandas as pd
import ast
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load the Datsets

In [190]:

def merge_csv_files_from_folder(folder_path):
    # Lista para almacenar los DataFrames
    df_list = []

    # Recorrer todos los archivos en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Obtener la ruta completa del archivo
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo CSV y agregarlo a la lista de DataFrames
            df = pd.read_csv(file_path)
            df_list.append(df)

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("No se encontraron archivos CSV en la carpeta.")
        return pd.DataFrame()  

data = merge_csv_files_from_folder("data_processed/csv/")
data.drop(columns="car_id", inplace=True)

C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\3516100075.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\3516100075.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\3516100075.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\3516100075.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\3516100075.py:12: DtypeWarning: Columns (40,41,44,45,46,47) have mixed types. Specify dtype op

In [191]:
def convert_boolean(value):
    if value == "True":
        return True
    elif value == "False":
        return False
    else:
        return value
for colum in [40,41,44,45,46,47]: data.iloc[:, colum] = data.iloc[:, colum].apply(lambda x: convert_boolean(x))

In [192]:
def find_dirty_columns(prefix): return [col for col in data.columns if prefix in col]

# Cleaning Dirty Columns

There are four columns that require cleaning: `^Confort_`, `^Acabado Interior_` (already cleaned), `^Acabado Exterior_`, and `^electricFeatures`. You need to select one of these columns and process the data until it becomes readable. All columns should carry meaningful information. If a column contains a wide range of phrases, you can merge them into a single column labeled as "description," which will allow the application of NLP techniques later.

### Guidelines for Cleaning:

- **Singular and Plural Forms**:  
  If two columns represent the same feature, but one is the plural form of the other (e.g., `Confort_Luz` and `Confort_Luces`), discard the plural column.

- **Duplicate Column Names**:  
  If two columns share the same name, you can discard one of them.

- **Numbered Columns**:  
  If multiple columns are part of a series ending with a number (e.g., `Confort_Dos`, `Confort_Uno`), they likely describe different aspects of a single feature. In this case, merge them into a single column.

- **Relevance of Columns**:  
  Most of the dirty columns have names that describe the feature of their information, but some columns may be irrelevant.

- **Example for Reference**:  
  The column `^Acabado Interior_` has already been cleaned and can serve as an example of how to approach the cleaning process.

- **Nan and "no tiene"**:  
  Some data has an Nan and "no tiene", the first one means that we don't know the data but probably you can deduce that feature with other columns, specially in electrical features, and the second the car does nor have that feature.

**Data Types**:
  BE CAREFULL most of the cells that contains " ['Asiento de... " is a string not a list
  
At the beginning of each section, you will find a printout of the columns that need to be cleaned.

---

Let me know if you need further revisions or clarification!


## ~~Profe~~ **Lingfeng** te hacia `ilu` escribir esto no ? `XD` 

o espera

Prof Lingfeng you enjoyed writing this, right? XD 

of course

## Functions

In [193]:
def remove_duplicates(lista, elemento): return [x for x in lista if x != elemento]

def check_unique_values(column):
    ret = sorted(set(data[column].to_list()))
    print(f"Column: {column}")
    for x in ret: print(f'{x}: {data[column].to_list().count(x)}')
    print(f'NA: {data[column].to_list().count(None)}')

## Clean Confort


In [194]:
confort_columns = find_dirty_columns("Confort")
for x in confort_columns: print(x)


Confort_Aire
Confort_Apertura
Confort_Aviso
Confort_Banda
Confort_Calefacción
Confort_Cierre
Confort_Compartimentos
Confort_Control
Confort_Dirección
Confort_Dos
Confort_Elevalunas
Confort_Faros
Confort_Indicador
Confort_Inmovilizador
Confort_Inmovilizador

Confort_Intermitentes
Confort_Lavafaros
Confort_Lavafaros

Confort_Limpiaparabrisas
Confort_Llave
Confort_Luces
Confort_Luz
Confort_Memoria
Confort_Protección
Confort_Regulación
Confort_Sistema
Confort_Tapa
Confort_Tarjeta
Confort_Techo
Confort_Testigo
Confort_Una
Confort_Volante


In [195]:
def classify_ac_zones(val):
    cleaned_val = re.sub(r"[\[\]']", '', val).strip().lower()

    match cleaned_val:
        case val if 'bizona' in val:
            return 'BIZONA'
        case val if 'trizona' in val or '3 zonas' in val:
            return 'TRIZONA'
        case val if '4 zonas' in val:
            return 'TETRAZONA'
        case _:
            return 'MONOZONA' 
        
def classify_ac_type(val): 
    cleaned_val = re.sub(r"[\[\]']", '', val).strip().lower()
    match cleaned_val:
        case val if 'semi-auto' in val or 'semi-automático' in val or 'semi-automatico' in val:
            return 'SEMIAUTO'
        case val if 'automático' in val or 'auto' in val or 'automatico' in val:
            return 'AUTO'
        case val if 'manual' in val:
            return 'MANUAL'
        case val if 'no tiene' in val:
            return 'NO_TIENE'
        case _:
            return 'STANDARD'  
        
def extraer_numero_tcinturones(text):
    if 'un' in text: return 1
    elif 'dos' in text:return 2
    elif 'tres' in text:return 3
    elif 'cuatro' in text:return 4
    elif 'cinco' in text:return 5
    elif 'seis' in text:return 6
    elif 'siete' in text:return 7
    elif 'Testigo' in text: return 2
    else: return 0
        

In [196]:
#Confort Aire
data["AC_Zones"] = data["Confort_Aire"].apply(classify_ac_zones)
data["AC_Type"] = data["Confort_Aire"].apply(classify_ac_type) 

#Confort_Apertura
#El maletero se abre a distancia? (Mecanicamente o electricamente)
data["trunk_auto_open"] = data["Confort_Apertura"].apply(lambda x: False if 'no tiene' in x else True)

#Se podría quitar la verdad
data["BandaTintada"] = data["Confort_Banda"].apply(lambda x: False if 'no tiene' in x else True)

data["Calefacción_Trasera"] = data["Confort_Calefacción"].apply(lambda x: False if 'no tiene' in x else True)


data['Doble_Cierre'] = data['Confort_Cierre'].apply(lambda x: True if 'doble cierre' in x else False)
data['Movil_NFC'] = data['Confort_Cierre'].apply(lambda x: True if 'teléfono móvil' in x or 'NFC' in x else False)
data['Metodo_Apertura'] = data['Confort_Cierre'].apply(
   lambda x: 'Mando a distancia' if 'mando a distancia' in x else
              'Tarjeta/llave inteligente' if 'tarjeta/llave inteligente' in x else
              'Llave' if 'llave' in x else
              'Teléfono móvil' if 'teléfono móvil' in x else
              'Desconocido'
)


data["Control_Crucero"] = data["Confort_Control"].apply(lambda x: True if 'crucero' in x else False)
data["Control_Crucero_Adaptativo"] = data["Confort_Control"].apply(lambda x: True if 'adaptativo' in x else False)  
data["SensoresDistancia"] = data["Confort_Control"].apply(lambda x: True if 'distancia' in x else False)
data["StopGo"] = data["Confort_Control"].apply(lambda x: True if 'stop' in x else False)


data["Dirección_Asistida"] = data["Confort_Dirección"].apply(lambda x: True if 'dirección asistida' in x.lower() else False)
data["Dirección_Electrica"] = data["Confort_Dirección"].apply(lambda x: True if 'eléctrica' in x.lower() else False)
data["Endurecimiento_Progresivo"] = data["Confort_Dirección"].apply(lambda x: True if 'endurecimiento progresivo' in x.lower() else False)
data["Desmultiplicacion_Variable"] = data["Confort_Dirección"].apply(lambda x: True if 'desmultiplicación variable' in x.lower() else False)
data["Dirección_Electro_Hidraulica"] = data["Confort_Dirección"].apply(lambda x: True if 'electro-hidráulica' in x.lower() else False)


data["Elevalunas_Electricos"] = data["Confort_Elevalunas"].apply(lambda x: True if 'elevalunas eléctricos' in x.lower() else False)
data["Un_Solo_Toque"] = data["Confort_Elevalunas"].apply(lambda x: True if 'uno de ellos de un solo toque' in x.lower() or 'dos de ellos de un solo toque' in x.lower() else False)
data["Elevalunas_Traseros"] = data["Confort_Elevalunas"].apply(lambda x: True if 'traseros' in x.lower() else False)


data["Tipo_Faro"] = data["Confort_Faros"].apply(lambda x: 
    'LED' if 'led' in x.lower() else
    'Xenón' if 'xenón' in x.lower() else
    'Halógeno' if 'halógeno' in x.lower() else
    'Otro')
data["Luz_Larga"] = data["Confort_Faros"].apply(lambda x: True if 'luz larga' in x.lower() else False)
data["Faros_Dobles"] = data["Confort_Faros"].apply(lambda x: True if 'dobles' in x.lower() else False)


data["Indicador_Baja_Presion_Neumaticos"] = data["Confort_Indicador"].apply(lambda x: 
    True if 'baja presion' in x.lower() else False)
data["Visualizacion_Presion"] = data["Confort_Indicador"].apply(lambda x: 
    True if 'visualización de presión' in x.lower() else False)
data["Indicador_Consumo"] = data["Confort_Indicador"].apply(lambda x: 
    True if 'indicador de consumo' in x.lower() else False)


data["Inmovilizador"] = data["Confort_Inmovilizador"].apply(lambda x: 
    True if 'Inmovilizador' in x else False)


data["Intermitentes_Laterales"] = data["Confort_Intermitentes"].apply(lambda x:
    True if 'laterales' in x else False)


data["Lavafaros"] = data["Confort_Lavafaros"].apply(lambda x:
    True if 'Lavafaros' in x else False)


data["Sensor_Lluvia"] = data["Confort_Limpiaparabrisas"].apply(lambda x: 
    True if 'sensor de lluvia' in x.lower() else False)
data["Intermitencia_Automatica"] = data["Confort_Limpiaparabrisas"].apply(lambda x: 
    True if 'intermitencia automática' in x.lower() else False)


data["Luces_Lectura"] = data["Confort_Luces"].apply(lambda x: 
    True if 'luces de lectura' in x.lower() else False)
data["Luces_Antiniebla"] = data["Confort_Luces"].apply(lambda x: 
    True if 'luces antiniebla' in x.lower() else False)
data["Luces_Cortesía"] = data["Confort_Luces"].apply(lambda x: 
    True if 'luces de cortesía' in x.lower() else False)
data["Luces_Laterales"] = data["Confort_Luces"].apply(lambda x: 
    True if 'luces laterales maniobras' in x.lower() else False)


data["Memoria_Asientos"] = data["Confort_Memoria"].apply(lambda x: 
    True if 'posición' in x else False)


data["Techo_Electrico"] = data["Confort_Techo"].apply(lambda x: 
    True if 'eléctrico' in x.lower() else False)
data["Techo_Inclinable"] = data["Confort_Techo"].apply(lambda x: 
    True if 'inclinable' in x.lower() else False)
data["Techo_Deslizante"] = data["Confort_Techo"].apply(lambda x: 
    True if 'deslizante' in x.lower() else False)
data["Control_Remoto"] = data["Confort_Techo"].apply(lambda x: 
    True if 'control remoto' in x.lower() else False)

data["Testigo_Cinturones"] = data["Confort_Testigo"].apply(lambda x: 
    True if 'testigo de cinturones' in x.lower() else False)

data["Numero_Testigos"] = data["Confort_Testigo"].apply(extraer_numero_tcinturones)

def process_confort_volante(x):
    if x == "no tiene": return x
    else:
        if len(ast.literal_eval(x)) > 1: return ". ".join(ast.literal_eval(x))
        else: return ast.literal_eval(x)[0]

# Feed para el modelo NLP
data["Confort_Volante_Descripción"] = data["Confort_Volante"].apply(process_confort_volante)


C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\2626097604.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Techo_Deslizante"] = data["Confort_Techo"].apply(lambda x:
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\2626097604.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Control_Remoto"] = data["Confort_Techo"].apply(lambda x:
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\2626097604.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

In [197]:
#Eliminar todas las columas con descripciones iniciales
for col in confort_columns: 
    data.drop(columns=col, inplace=True)

## Clean Acabado Interior 

In [198]:
acabado_interior_columns = find_dirty_columns("Acabado Interior")

In [199]:
acabado_interior_columns.remove("Acabado Interior_Asientos")
data.drop(columns="Acabado Interior_Asientos", inplace=True)

#for x in acabado_interior_columns: print(x)
#data[acabado_interior_columns]

In [200]:
# Definir funciones personalizadas para la conversión
def convert_sports_pedals(value):
    if value == "['Pedales deportivos']":
        return True
    elif value == 'no tiene':
        return False
    else:
        return None  # Por si hay otros valores

def convert_interior_clock(value):
    if value == "['Reloj analógico']":
        return 'analogico'
    elif value == "['Reloj digital']":
        return 'digital'
    elif value == "['Reloj']":
        return 'regular'
    elif value == "no tiene":
        return 'no tiene'
    else:
        return None  # Por si hay otros valores

# Aplicar las funciones a las columnas correspondientes
data["sports_pedals"] = data["Acabado Interior_Pedales"].apply(convert_sports_pedals)
data["interior_clock"] = data["Acabado Interior_Reloj"].apply(convert_interior_clock)

#check_unique_values("sports_pedals")
#check_unique_values("interior_clock")


C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\950979874.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["sports_pedals"] = data["Acabado Interior_Pedales"].apply(convert_sports_pedals)
C:\Users\lingf\AppData\Local\Temp\ipykernel_22236\950979874.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["interior_clock"] = data["Acabado Interior_Reloj"].apply(convert_interior_clock)


In [201]:

def process_arm_suppport_front(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('delantero' in text): return ("no tiene")
    text = [word for word in text.split() if word not in stop_words]
    if '#' in text: 
        ret = " ".join( text[ (text.index('delantero')+1):text.index('#')] )
        return "Tiene" if len(ret) == 0 else ret
    else: 
        ret = " ".join( text[(text.index('delantero')+1):len(text)] ) 
        return "Tiene" if len(ret) == 0 else ret

def process_arm_suppport_back(phrases: list) -> str:
    stop_words = set(stopwords.words('spanish'))
    stop_words.add(",")
    text = " # ".join(phrases)
    if not ('trasero' in text): return ("no tiene")

    text = [word for word in text.split() if word not in stop_words]

    ret = " ".join( text[(text.index('trasero')+1):len(text)] ) 
    return "Tiene" if len(ret) == 0 else ret 

arm_suppport_front = data["Acabado Interior_Apoyabrazos"].apply(lambda x: x if x == 'no tiene' else process_arm_suppport_front(eval(x)) )
arm_suppport_back = data["Acabado Interior_Apoyabrazos"].apply(lambda x: x if x == 'no tiene' else process_arm_suppport_back(eval(x)) )

data = pd.concat([data, arm_suppport_front.rename("arm_suppport_front"), arm_suppport_back.rename("arm_suppport_back")], axis=1)

check_unique_values("arm_suppport_front")
check_unique_values("arm_suppport_back")

Column: arm_suppport_front
Tiene: 117241
abatible: 98
ajustable: 1
caja integrada: 4238
no tiene: 43859
NA: 0
Column: arm_suppport_back
Tiene: 64993
abatible: 5
acceso maletero: 406
ajustable: 2
caja integrada: 1415
caja integrada acceso maletero: 559
no tiene: 98057
NA: 0


In [202]:
def process_seat_description(phrases: list) -> str: return ". ".join(phrases)

data["seat_description"] = data["Acabado Interior_Asiento"].apply( lambda x: x if x == 'no tiene' else process_seat_description(eval(x)) )

In [203]:
data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].apply( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].apply( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].apply( lambda x: "['no tiene']" if x == 'no tiene' else x )
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].apply( lambda x: "['no tiene']" if x == 'no tiene' else x )

data["Acabado Interior_Uno"] = data["Acabado Interior_Uno"].apply(ast.literal_eval)
data["Acabado Interior_Dos"] = data["Acabado Interior_Dos"].apply(ast.literal_eval)
data["Acabado Interior_Tres"] = data["Acabado Interior_Tres"].apply(ast.literal_eval)
data["Acabado Interior_Reposacabezas"] = data["Acabado Interior_Reposacabezas"].apply(ast.literal_eval)

In [204]:
data['head_supporter_description'] = data.apply(
    lambda row: ". ".join(
        remove_duplicates(row["Acabado Interior_Uno"] + row["Acabado Interior_Dos"] + row["Acabado Interior_Tres"] + row["Acabado Interior_Reposacabezas"], 'no tiene')
    ), 
    axis=1)
#check_unique_values("head_supporter")

data["transmissionTypeId"] = data["transmissionTypeId"].apply(lambda x: "Manual" if x == 2 else "Automatico")

for col in acabado_interior_columns: data.drop(columns= col, inplace=True)

In [205]:
print(len(data))
data = data[~data['airbag'].apply(lambda x: isinstance(x, float))]
print(len(data))
data["airbag_front"] = data["airbag"].apply(lambda x: True if "Airbag frontal" in x else False)
data["airbag_side_window"] = data["airbag"].apply(lambda x: True if "Airbag lateral de cortina" in x else False)
data["airbags_side"] = data["airbag"].apply(lambda x: True if "Airbags laterales" in x else False)

data.drop(columns= "airbag", inplace=True)

165437
165416


In [206]:
data.loc[data["fuelType"] == "Eléctrico", "motor_description"] = "no tiene"

In [210]:
print(len(data))
data = data[~data['motor_description'].apply(lambda x: isinstance(x, float))]
print(len(data))

print(data["fuelType"].value_counts())
# Dictionary to convert Spanish words to numbers
word_to_number = {
    "uno": "1", "dos": "2", "tres": "3", "cuatro": "4", "cinco": "5",
    "seis": "6", "siete": "7", "ocho": "8", "nueve": "9", "diez": "10",
    "once": "11", "doce": "12", "trece": "13", "catorce": "14", "quince": "15",
    "dieciséis": "16", "diecisiete": "17", "dieciocho": "18", "diecinueve": "19", "veinte": "20", 
    "viente": "20", "vientecinco": "25", "vientecuatro": "24", "vienteuno": "21", "vientedos": "22", 
    "vientetres": "23", "vienteseis": "26", "vientesiete": "27", "vienteocho": "28", "vientenueve": "29", 
    "treinta": "30", 
       
}

# 1. Function to extract engine displacement in liters (e.g., "1.6 liters")
def extract_displacement_liters(phrase):
    match = re.search(r"(\d+,\d+) litros", phrase)
    return match.group(1) if match else "NA"

# 2. Function to extract engine displacement in cubic centimeters (e.g., "1.582 cc")
def extract_displacement_cc(phrase):
    match = re.search(r"\(\s*([\d,.]+)\s*cc\s*\)", phrase)
    return match.group(1) if match else "NA"

# 3. Function to extract the number of cylinders (e.g., "four cylinders" or "4 cylinders")
def extract_number_of_cylinders(phrase):
    match = re.search(r"(\w+|\d+) cilindros", phrase)
    if match:
        cylinders = match.group(1).lower()
        return word_to_number.get(cylinders, cylinders)  # Convert word to number if necessary
    return "NA"

# 4. Function to extract the number of valves per cylinder (e.g., "four valves per cylinder" or "4 valves")
def extract_valves_per_cylinder(phrase):
    match = re.search(r"(\w+|\d+) válvulas por cilindro", phrase)
    if match:
        valves = match.group(1).lower()
        return word_to_number.get(valves, valves)  # Convert word to number if necessary
    return "NA"

# 5. Function to extract cylinder diameter (e.g., "77.2 mm")
def extract_bore_diameter(phrase):
    match = re.search(r"(\d+,\d+)\s*mm de diámetro", phrase)
    return match.group(1) if match else "NA"

# 6. Function to extract stroke length (e.g., "84.5 mm")
def extract_stroke_length(phrase):
    match = re.search(r"(\d+,\d+)\s*mm de carrera", phrase)
    return match.group(1) if match else "NA"

# 7. Function to extract compression ratio (e.g., "17.3")
def extract_compression_ratio(phrase):
    match = re.search(r"relación de compresión:\s*(\d+,\d+)", phrase)
    return match.group(1) if match else "NA"


data["displacement_liters"] = data["motor_description"].apply(extract_displacement_liters)
#print(data["displacement_liters"].value_counts())

data["displacement_cc"] = data["motor_description"].apply(extract_displacement_cc)
#print(data["displacement_cc"].value_counts())

data["number_of_cylinders"] = data["motor_description"].apply(extract_number_of_cylinders)
#print(data["number_of_cylinders"].value_counts())

data["valves_per_cylinder"] = data["motor_description"].apply(extract_valves_per_cylinder)
#print(data["valves_per_cylinder"].value_counts())

data["bore_diameter"] = data["motor_description"].apply(extract_bore_diameter)
#print(data["bore_diameter"].value_counts())

data["stroke_length"] = data["motor_description"].apply(extract_stroke_length)
#print(data["stroke_length"].value_counts())

data["compression_ratio"] = data["motor_description"].apply(extract_compression_ratio)
#print(data["compression_ratio"].value_counts())



165141
165141
fuelType
Diésel                71808
Gasolina              57396
Híbrido               20099
Híbrido enchufable     7955
Eléctrico              6591
Gas licuado (GLP)       930
Gas natural (CNG)       360
Name: count, dtype: int64


In [211]:
print(len(data))
data = data[~data['speakers'].apply(lambda x: isinstance(x, float))]
print(len(data))
data["speakers"] = data["speakers"].apply(lambda x: word_to_number[x.lower()] if x != "0" else 0)

165141
165141


## Clean Acabado Exterior

In [212]:
stopw = frozenset(stopwords.words('spanish')) | {","}

def cleanTry(phrases: list) -> str:
    if not phrases:
        return "no tiene"
    
    text = phrases
    
    text = [word for word in text.split() if word.lower() not in stopw]

    return "no tiene" if len(text) == 0 else " ".join(text)

def extract_diameter(text):
    match = re.search(r'(\d+(?:,\d+)?)\s*pulgadas\s*diámetro', text)
    if match:
        return (float(match.group(1).replace(',', '.'))*2.54)
    else:
        return "no tiene"  
    
def clean_acabado(cell):
    cell_cleaned = re.sub(r"[\[\]']", '', cell)
    cell_cleaned = cell_cleaned.strip().lower()
    cell_cleaned = re.sub(r'\s+', ' ', cell_cleaned)
    return cell_cleaned


In [213]:
data.drop(columns=["Acabado Exterior_Maletero","Acabado Exterior_Pasos","Acabado Exterior_Maletero","Acabado Exterior_Defensa","Acabado Exterior_Guardabarros"], inplace=True)

In [214]:
acabado_exterior_columns = find_dirty_columns("Acabado Exterior")

data[acabado_exterior_columns] = data[acabado_exterior_columns].apply(lambda col: col.apply(lambda x: clean_acabado(x) if x != 'no tiene' else x))

data[acabado_exterior_columns] = data[acabado_exterior_columns].apply(lambda col: col.apply(lambda x: cleanTry(x) if x != 'no tiene' else x))

data["Llantas_Diametro_cm"] = data["Acabado Exterior_Llantas"].apply(lambda x: extract_diameter(x))

acabado_exterior_columns.append("Llantas_Diametro_cm")



for x in acabado_exterior_columns:
    
    #print(x)
    print(data[x].nbytes)
    print("Descripcion --------------------------")
    print(data[x].astype('category').nbytes)
    #print(data[x].describe())
    #print("Counts--------------------------------")
    #print(data[x].value_counts())
    #print("CleanTry------------------------------")
    

1321128
Descripcion --------------------------
165429
1321128
Descripcion --------------------------
165245
1321128
Descripcion --------------------------
165157
1321128
Descripcion --------------------------
344706
1321128
Descripcion --------------------------
165285
1321128
Descripcion --------------------------
368882
1321128
Descripcion --------------------------
165421
1321128
Descripcion --------------------------
331538
1321128
Descripcion --------------------------
333874
1321128
Descripcion --------------------------
165565
1321128
Descripcion --------------------------
165253


## Electronic Features

In [215]:
print(len(data))
data = data[~data['fuelType'].apply(lambda x: isinstance(x, float))]
print(len(data))


check_unique_values("fuelType")

electronic_columns = find_dirty_columns("electric")
#for x in electronic_columns: print(x)


165141
165139
Column: fuelType
Diésel: 71808
Eléctrico: 6591
Gas licuado (GLP): 930
Gas natural (CNG): 360
Gasolina: 57396
Híbrido: 20099
Híbrido enchufable: 7955
NA: 0


In [216]:
# Seleccionar las columnas específicas del dataset
# Mostrar las primeras filas de esas columnas
data[electronic_columns].head()


,electricFeatures.powerSource.batteryType,electricFeatures.powerSource.batteryVoltage.value,electricFeatures.powerSource.batteryVoltage.unit,electricFeatures.powerSource.maximumBatteryKWH.value,electricFeatures.powerSource.maximumBatteryKWH.unit,electricFeatures.power.maxPower.value,electricFeatures.power.maxPower.unit,electricFeatures.chargingInformation.standardMode.scenario,electricFeatures.chargingInformation.standardMode.duration.value,electricFeatures.chargingInformation.standardMode.duration.unit,...,electricFeatures.chargingInformation.standardMode.chargeEnd.value,electricFeatures.chargingInformation.standardMode.chargeEnd.unit,electricFeatures.chargingInformation.fastMode.chargeStart.value,electricFeatures.chargingInformation.fastMode.chargeStart.unit,electricFeatures.chargingInformation.fastMode.voltage.value,electricFeatures.chargingInformation.fastMode.voltage.unit,electricFeatures.chargingInformation.standardMode.maxKW.value,electricFeatures.chargingInformation.standardMode.maxKW.unit,electricFeatures.chargingInformation.fastMode.amps.value,electricFeatures.chargingInformation.fastMode.amps.unit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iones de litio,352.0,V,42.24,kWh,118.0,CV,modo 2,915.0,minutos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
def clean_and_transform_data(df):
    df_cleaned = df.copy()

    # Lista de columnas que queremos eliminar
    columns_to_drop = [
        'electricFeatures.chargingInformation.standardMode.chargeEnd.value',
        'electricFeatures.chargingInformation.standardMode.chargeEnd.unit',
        'electricFeatures.powerSource.batteryVoltage.value',
        'electricFeatures.powerSource.batteryVoltage.unit',
        'electricFeatures.powerSource.maximumBatteryKWH.value',
        'electricFeatures.powerSource.maximumBatteryKWH.unit',
        'electricFeatures.power.maxPower.value',
        'electricFeatures.power.maxPower.unit',
        'electricFeatures.chargingInformation.standardMode.duration.value',
        'electricFeatures.chargingInformation.standardMode.duration.unit',
        'electricFeatures.chargingInformation.fastMode.duration.value',
        'electricFeatures.chargingInformation.fastMode.duration.unit',
        'electricFeatures.chargingInformation.fastMode.maxKW.value',
        'electricFeatures.chargingInformation.fastMode.maxKW.unit',
        'electricFeatures.chargingInformation.fastMode.chargeEnd.value',
        'electricFeatures.chargingInformation.fastMode.chargeEnd.unit',
        'electricFeatures.combinedConsumption.value',
        'electricFeatures.combinedConsumption.unit',
        'electricFeatures.powerSource.onboardCharger.value',
        'electricFeatures.powerSource.onboardCharger.unit',
        'electricFeatures.chargingInformation.standardMode.voltage.value',
        'electricFeatures.chargingInformation.standardMode.voltage.unit',
        'electricFeatures.chargingInformation.standardMode.chargeStart.value',
        'electricFeatures.chargingInformation.standardMode.chargeStart.unit',
        'electricFeatures.chargingInformation.fastMode.chargeStart.value',
        'electricFeatures.chargingInformation.fastMode.chargeStart.unit',
        'electricFeatures.chargingInformation.fastMode.voltage.value',
        'electricFeatures.chargingInformation.fastMode.voltage.unit',
        'electricFeatures.chargingInformation.standardMode.maxKW.value',
        'electricFeatures.chargingInformation.standardMode.maxKW.unit',
        'electricFeatures.chargingInformation.fastMode.amps.value',
        'electricFeatures.chargingInformation.fastMode.amps.unit',
        'electricFeatures.chargingInformation.standardMode.amps.value',
        'electricFeatures.chargingInformation.standardMode.amps.unit',
        'electricFeatures.range.value',
        'electricFeatures.range.unit',
        'electricFeatures.chargingInformation.standardMode.scenario',
        'electricFeatures.chargingInformation.fastMode.scenario'


    ]

    # Combinación de columnas 'value' y 'unit' evitando NaN
    # Battery Voltage
    if 'electricFeatures.powerSource.batteryVoltage.value' in df_cleaned.columns and \
       'electricFeatures.powerSource.batteryVoltage.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.powerSource.batteryVoltage.value'].notna() & \
               df_cleaned['electricFeatures.powerSource.batteryVoltage.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.batteryVoltage'] = df_cleaned['electricFeatures.powerSource.batteryVoltage.value'].astype(str) + ' ' + df_cleaned['electricFeatures.powerSource.batteryVoltage.unit']

    if 'electricFeatures.range.value' in df_cleaned.columns and \
       'electricFeatures.range.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.range.value'].notna() & \
               df_cleaned['electricFeatures.range.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.range'] = df_cleaned['electricFeatures.range.value'].astype(str) + ' ' + df_cleaned['electricFeatures.range.unit']

    # Maximum Battery KWH
    if 'electricFeatures.powerSource.maximumBatteryKWH.value' in df_cleaned.columns and \
       'electricFeatures.powerSource.maximumBatteryKWH.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.powerSource.maximumBatteryKWH.value'].notna() & \
               df_cleaned['electricFeatures.powerSource.maximumBatteryKWH.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.maximumBatteryKWH'] = df_cleaned['electricFeatures.powerSource.maximumBatteryKWH.value'].astype(str) + ' ' + df_cleaned['electricFeatures.powerSource.maximumBatteryKWH.unit']

    # Power Max
    if 'electricFeatures.power.maxPower.value' in df_cleaned.columns and \
       'electricFeatures.power.maxPower.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.power.maxPower.value'].notna() & \
               df_cleaned['electricFeatures.power.maxPower.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.maxPower'] = df_cleaned['electricFeatures.power.maxPower.value'].astype(str) + ' ' + df_cleaned['electricFeatures.power.maxPower.unit']

    # Standard Mode Duration
    if 'electricFeatures.chargingInformation.standardMode.duration.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.duration.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.duration.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.duration.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.standardChargeDuration'] = df_cleaned['electricFeatures.chargingInformation.standardMode.duration.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.duration.unit']

    # Fast Mode Duration
    if 'electricFeatures.chargingInformation.fastMode.duration.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.duration.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.duration.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.duration.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastChargeDuration'] = df_cleaned['electricFeatures.chargingInformation.fastMode.duration.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.duration.unit']

    # Fast Mode Max KW
    if 'electricFeatures.chargingInformation.fastMode.maxKW.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.maxKW.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.maxKW.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.maxKW.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastMaxKW'] = df_cleaned['electricFeatures.chargingInformation.fastMode.maxKW.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.maxKW.unit']

    # Fast Mode Charge End
    if 'electricFeatures.chargingInformation.fastMode.chargeEnd.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.chargeEnd.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.chargeEnd.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.chargeEnd.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastChargeEnd'] = df_cleaned['electricFeatures.chargingInformation.fastMode.chargeEnd.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.chargeEnd.unit']

    # Combined Consumption
    if 'electricFeatures.combinedConsumption.value' in df_cleaned.columns and \
       'electricFeatures.combinedConsumption.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.combinedConsumption.value'].notna() & \
               df_cleaned['electricFeatures.combinedConsumption.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.combinedConsumption'] = df_cleaned['electricFeatures.combinedConsumption.value'].astype(str) + ' ' + df_cleaned['electricFeatures.combinedConsumption.unit']

    # Onboard Charger
    if 'electricFeatures.powerSource.onboardCharger.value' in df_cleaned.columns and \
       'electricFeatures.powerSource.onboardCharger.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.powerSource.onboardCharger.value'].notna() & \
               df_cleaned['electricFeatures.powerSource.onboardCharger.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.onboardCharger'] = df_cleaned['electricFeatures.powerSource.onboardCharger.value'].astype(str) + ' ' + df_cleaned['electricFeatures.powerSource.onboardCharger.unit']

    # Standard Mode Voltage
    if 'electricFeatures.chargingInformation.standardMode.voltage.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.voltage.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.voltage.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.voltage.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.standardModeVoltage'] = df_cleaned['electricFeatures.chargingInformation.standardMode.voltage.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.voltage.unit']

    # Standard Mode Charge Start
    if 'electricFeatures.chargingInformation.standardMode.chargeStart.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.chargeStart.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.chargeStart.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.chargeStart.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.standardModeChargeStart'] = df_cleaned['electricFeatures.chargingInformation.standardMode.chargeStart.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.chargeStart.unit']

    # Standard Mode Charge End
    if 'electricFeatures.chargingInformation.standardMode.chargeEnd.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.chargeEnd.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.chargeEnd.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.chargeEnd.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.standardModeChargeEnd'] = df_cleaned['electricFeatures.chargingInformation.standardMode.chargeEnd.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.chargeEnd.unit']

    # Fast Mode Charge Start
    if 'electricFeatures.chargingInformation.fastMode.chargeStart.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.chargeStart.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.chargeStart.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.chargeStart.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastModeChargeStart'] = df_cleaned['electricFeatures.chargingInformation.fastMode.chargeStart.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.chargeStart.unit']

    # Fast Mode Voltage
    if 'electricFeatures.chargingInformation.fastMode.voltage.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.voltage.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.voltage.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.voltage.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastModeVoltage'] = df_cleaned['electricFeatures.chargingInformation.fastMode.voltage.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.voltage.unit']

    # Standard Mode Max KW
    if 'electricFeatures.chargingInformation.standardMode.maxKW.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.maxKW.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.maxKW.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.maxKW.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.standardModeMaxKW'] = df_cleaned['electricFeatures.chargingInformation.standardMode.maxKW.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.maxKW.unit']

    # Fast Mode Amps
    if 'electricFeatures.chargingInformation.fastMode.amps.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.fastMode.amps.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.fastMode.amps.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.fastMode.amps.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.fastModeAmps'] = df_cleaned['electricFeatures.chargingInformation.fastMode.amps.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.fastMode.amps.unit']


    if 'electricFeatures.chargingInformation.standardMode.amps.value' in df_cleaned.columns and \
       'electricFeatures.chargingInformation.standardMode.amps.unit' in df_cleaned.columns:
        mask = df_cleaned['electricFeatures.chargingInformation.standardMode.amps.value'].notna() & \
               df_cleaned['electricFeatures.chargingInformation.standardMode.amps.unit'].notna()
        df_cleaned.loc[mask, 'electricFeatures.chargingInformation.standardMode'] = df_cleaned['electricFeatures.chargingInformation.standardMode.amps.value'].astype(str) + ' ' + df_cleaned['electricFeatures.chargingInformation.standardMode.amps.unit']

    # Eliminar solo las columnas que existen en el dataset
    columns_to_drop = [col for col in columns_to_drop if col in df_cleaned.columns]
    df_cleaned.drop(columns=columns_to_drop, inplace=True)

    return df_cleaned

# Limpiar el dataset
data = clean_and_transform_data(data)

# Mostrar una vista previa de los primeros registros
data.head()


,price,km,year,color,cubicCapacity,brand,model,version,fuelType,bodyTypeId,...,electricFeatures.combinedConsumption,electricFeatures.onboardCharger,electricFeatures.standardModeVoltage,electricFeatures.standardModeChargeStart,electricFeatures.standardModeChargeEnd,electricFeatures.fastModeChargeStart,electricFeatures.fastModeVoltage,electricFeatures.standardModeMaxKW,electricFeatures.fastModeAmps,electricFeatures.chargingInformation.standardMode
0,10600,99700.0,2006.0,Negro,1798.0,MAZDA,MX-5,Active 1.8,Gasolina,Cabrio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12450,91924.0,2015.0,Rojo (*TLV*),1362.0,OPEL,Cabrio,1.4 T SS Excellence,Gasolina,Cabrio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,134900,62000.0,2015.0,Negro,3800.0,PORSCHE,911,Turbo Coupe,Gasolina,Coupe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27500,13500.0,2022.0,Rosa,NaN,FIAT,500,laPrima 31 320km 85kW 118CV,Eléctrico,Berlina,...,14.6 kWh/100km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0 A
4,27990,13559.0,2023.0,Blanco,1498.0,CUPRA,León,1.5 eTSI 110kW 150CV DSG,Híbrido,Berlina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
def ver_columna(df, nombre_columna):
    if nombre_columna in df.columns:
        print(df[nombre_columna].head())  # Mostrar las primeras filas de la columna
    else:
        print(f"La columna '{nombre_columna}' no existe en el DataFrame.")

# Ejemplo de uso
ver_columna(data, 'electricFeatures.chargingInformation.standardMode.chargeEnd.value')


La columna 'electricFeatures.chargingInformation.standardMode.chargeEnd.value' no existe en el DataFrame.


In [219]:
def find_dirty_columns2(prefix): return [col for col in data.columns if prefix in col]

In [220]:
def count_nan_and_non_nan(df, column_name):
    if column_name in df.columns:
        # Contar valores no nulos
        non_nan_count = df[column_name].notna().sum()
        # Contar valores NaN
        nan_count = df[column_name].isna().sum()
        print(f"Columna: {column_name}")
        print(f"Valores normales: {non_nan_count}")
        print(f"Valores NaN: {nan_count}")
    else:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        
# Usar la función con el DataFrame y la columna que desees
column_to_check = 'electricFeatures.chargingInformation.standardMode.chargeEnd.value'  # Cambia por la columna que desees
count_nan_and_non_nan(data, column_to_check)


La columna 'electricFeatures.chargingInformation.standardMode.chargeEnd.value' no existe en el DataFrame.


In [221]:
electronic_columns2 = find_dirty_columns2("electric")
#for x in electronic_columns2: print(x)
# Mostrar las primeras filas de esas columnas
data[electronic_columns2].head()

,electricFeatures.powerSource.batteryType,electricFeatures.motorType,electricFeatures.chargingConnector,electricFeatures.combinedConsumption.testType,electricFeatures.batteryVoltage,electricFeatures.range,electricFeatures.maximumBatteryKWH,electricFeatures.maxPower,electricFeatures.standardChargeDuration,electricFeatures.fastChargeDuration,...,electricFeatures.combinedConsumption,electricFeatures.onboardCharger,electricFeatures.standardModeVoltage,electricFeatures.standardModeChargeStart,electricFeatures.standardModeChargeEnd,electricFeatures.fastModeChargeStart,electricFeatures.fastModeVoltage,electricFeatures.standardModeMaxKW,electricFeatures.fastModeAmps,electricFeatures.chargingInformation.standardMode
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iones de litio,['motor síncrono de imanes permanentes CA'],"['CCS(Combo)', 'Tipo 2 Mennekes']",BEV/HEV consumo de energía eléctrica,352.0 V,305.0 KM,42.24 kWh,118.0 CV,915.0 minutos,34.0 minutos,...,14.6 kWh/100km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0 A
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save file

In [222]:
data = data.rename(columns={'parking_sensors': 'parking_sensors_description'})
data = data.rename(columns={'Acabado Exterior_Llantas': 'Exterior_Llantas_description'})
data = data.rename(columns={'Acabado Exterior_Molduras': 'Exterior_Molduras_description'})
data = data.rename(columns={'Acabado Exterior_Neumáticos': 'Exterior_Neumáticos_description'})
data = data.rename(columns={'Acabado Exterior_Puerta': 'Exterior_Puerta_description'})
data = data.rename(columns={'Acabado Exterior_Retrovisor': 'Exterior_Retrovisor_description'})
data = data.rename(columns={'Confort_Volante_Descripción': 'Confort_Volante_description'})

In [232]:
max_file_size = 100 * 1024 * 1024
def split_dataframe_to_csv(df, max_file_size, output_folder ,base_filename):
    file_index = 1
    chunk_size = len(df) // 7 
    while chunk_size > 0:
        start_row = 0
        while start_row < len(df):
            end_row = start_row + chunk_size
            chunk = df.iloc[start_row:end_row]
            filename = f"{output_folder}/{base_filename}_part{file_index}.csv"
            chunk.to_csv(filename, index=False)
            if os.path.getsize(filename) > max_file_size:
                chunk_size //= 2
                os.remove(filename)
            else:
                start_row = end_row
                file_index += 1

        if start_row >= len(df):
            break

train_set, test_set = train_test_split(data, test_size=0.3, random_state=42, stratify=data['fuelType'])

print("Original fuelType distribution:\n", data['fuelType'].value_counts(normalize=True))
print("\nTrain set fuelType distribution:\n", train_set['fuelType'].value_counts(normalize=True))
print("\nTest set fuelType distribution:\n", test_set['fuelType'].value_counts(normalize=True))

split_dataframe_to_csv(data, max_file_size, "final_data/original", "original")
split_dataframe_to_csv(train_set, max_file_size, "final_data/train", "train")
split_dataframe_to_csv(test_set, max_file_size, "final_data/test", "test")

Original fuelType distribution:
 fuelType
Diésel                0.434834
Gasolina              0.347562
Híbrido               0.121710
Híbrido enchufable    0.048172
Eléctrico             0.039912
Gas licuado (GLP)     0.005632
Gas natural (CNG)     0.002180
Name: proportion, dtype: float64

Train set fuelType distribution:
 fuelType
Diésel                0.434830
Gasolina              0.347561
Híbrido               0.121707
Híbrido enchufable    0.048176
Eléctrico             0.039915
Gas licuado (GLP)     0.005632
Gas natural (CNG)     0.002180
Name: proportion, dtype: float64

Test set fuelType distribution:
 fuelType
Diésel                0.434843
Gasolina              0.347564
Híbrido               0.121715
Híbrido enchufable    0.048161
Eléctrico             0.039906
Gas licuado (GLP)     0.005632
Gas natural (CNG)     0.002180
Name: proportion, dtype: float64
